In [1]:
import os
import sys
sys.path.insert(0, '../PYTHON/HLB')
import orange_canopy_tif as orca
import gdalbasics as gdb

from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

from imgaug import augmenters as iaa

import numpy as np

import skimage

logs = '../Mask_RCNN/logs'
dataset = '../Mask_RCNN/datasets/canopy'

## Treinamento

In [2]:
config = orca.OrangeCanopyConfig()
# config.display()

In [3]:
weights = '../Mask_RCNN/mask_rcnn_coco.h5'
# weights = '../Mask_RCNN/logs/trees.h5'

In [ ]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=logs)

In [ ]:
model.load_weights(weights, 
                   by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
train_dataset = orca.OrangeCanopyDataset()
train_dataset.load_data(dataset, 'train')
train_dataset.prepare()

val_dataset = orca.OrangeCanopyDataset()
val_dataset.load_data(dataset, 'val')
val_dataset.prepare()

augmentation = iaa.SomeOf((0, 2), [iaa.Fliplr(0.5), iaa.Flipud(0.5), iaa.OneOf([iaa.Affine(rotate=90), iaa.Affine(rotate=180), iaa.Affine(rotate=270)]), iaa.Multiply((0.5, 1.5))])

In [ ]:
model.train(train_dataset, val_dataset,
            learning_rate=config.LEARNING_RATE,
            epochs=4,
            augmentation=augmentation,
            layers='heads')

# Inferência

In [4]:
class InferenceConfig(orca.OrangeCanopyConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # I kept the same configuration of the training
    NUM_CLASSES = 1 + 1
    NAME = 'trees'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    BACKBONE = "resnet50"
    DETECTION_MIN_CONFIDENCE = 0.5
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64)
    USE_MINI_MASK = False
    IMAGE_CHANNEL_COUNT = 3
    RPN_NMS_THRESHOLD = 0.9
    MEAN_PIXEL = np.array([105, 236, 189])
    DETECTION_NMS_THRESHOLD = 0.3
    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    TRAIN_ROIS_PER_IMAGE = 100
    RPN_TRAIN_ANCHORS_PER_IMAGE = 100
    
config = InferenceConfig()

In [5]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Instructions for updating:
Use fn_output_signature instead


In [6]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20210803T1653/mask_rcnn_oranges_trees_canopy_0012.h5'

model.load_weights(weights, by_name=True)

Re-starting from epoch 12


In [46]:
images_test_dir = os.path.join(dataset, 'real_test', 'images', 'TIF')
# images_test_dir = '../IMGS'

image_paths = []
for filename in os.listdir(images_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.tif']:
        image_paths.append(os.path.join(images_test_dir, filename))


shape_path_aux = dataset + '/results/'
temp_path_prediction = dataset + '/results/' + 'TEMP_RESP.tif'
shape_prediction_name = 'pred_image'
shape_prediction_ext = '.shp'

for path in image_paths:
    array, image_tif = gdb.readimagetif(path, 'Integer')
    result = model.detect([array[:,:,0:3]], verbose=1)[0]
    masks = result['masks']
    dim_masks = masks.shape
    masks_final = np.zeros((dim_masks[0], dim_masks[1]), dtype=np.int32)
    for cont_mask in range(dim_masks[2]):
        aux = masks[:, :, cont_mask].copy()
        for j in range(aux.shape[0]):
            for i in range(aux.shape[1]):
                if aux[j, i] != 0:
                    masks_final[j, i] = (cont_mask + 1)
                # end if
            # end for
        # end for
    # end for

    # save the shape
    origin_x, pixel_width, rot_x, origin_y, rot_y, pixel_height = image_tif.GetGeoTransform()
    drive = image_tif.GetDriver()
    projection = image_tif.GetProjection()
    raster_origin = (origin_y, origin_x)
    gdb.array2raster(temp_path_prediction, 'Integer', raster_origin, pixel_height, pixel_width, rot_y, rot_x, drive, projection, masks_final)
    shape_path = shape_path_aux + shape_prediction_name + str(path[-8:-4]) + shape_prediction_ext
    gdb.raster2polygon(temp_path_prediction, shape_path, 'real_test')
    masks_final = None
    break

Processing 1 images
image                    shape: (256, 256, 3)         min:    0.00000  max:  253.00000  uint16
molded_images            shape: (1, 256, 256, 3)      min: -233.00000  max:  147.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -0.17747  max:    0.99541  float32


In [ ]:
visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'],
                                val_dataset.class_names, r['scores'], figsize=(15, 15))

In [45]:
shape_path

'../Mask_RCNN/datasets/canopy/result/pred_image0342.shp'